In [2]:
from tensorflow import keras
from component.vocal_cords import get_vocal_cords_model
from utils.feature_extraction import  get_MFCCS
import numpy as np

In [3]:
import random
class DataGenerator(keras.utils.Sequence):
    def __init__(self, X, y, batch_size):
        self.X = X
        self.y = y
        self.batch_size = batch_size  
        self.indexes = set(np.arange(len(X)))
        
    def __len__(self):
        return len(self.indexes)

    def __getitem__(self, _):
        indexes = random.sample(self.indexes, self.batch_size)
        X = []
        y = []
        for index in indexes:
            X.append(get_MFCCS(self.X[index]))
            y.append(self.y[index])
        return np.array(X), np.array(y, dtype=int)
    

In [4]:
import glob
X1 = glob.glob("audio/with_them/*.wav")
X0 = glob.glob("audio/without_them/*.wav")
X = X1 + X0
y = [1]*len(X1) + [0] * len(X0)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, random_state=69, stratify=y)

In [6]:
train_generator = DataGenerator(X_train, y_train, 8)
val_generator = DataGenerator(X_val, y_val, 8)

In [7]:
model = get_vocal_cords_model(input_shape=(300,200,1))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['acc'])

In [8]:
history = model.fit(train_generator, steps_per_epoch=30, validation_steps=50, epochs=50, validation_data=val_generator,)
          # callbacks=[keras.callbacks.EarlyStopping(monitor='val_acc', patience=3)])

Epoch 1/50
 9/30 [========>.....................] - ETA: 4s - loss: 1.3761 - acc: 0.4869  

KeyboardInterrupt: 

In [9]:
model.save_weights("vocal_cords_v1.h5")